In [2]:
import requests
from flask import Flask, request, jsonify, send_from_directory
app = Flask(__name__)
import pandas as pd
import quandl
import math
import random
import os
import datetime
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import numpy as np

c:\users\hp\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Needed Fields
['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']

In [3]:
dataFile = 'stockData.csv';
data = pd.read_csv(dataFile,parse_dates=True)

In [4]:
list(data.columns.values)

['#',
 'DATE',
 'TRADING CODE',
 'LTP*',
 'HIGH',
 'LOW',
 'OPENP*',
 'CLOSEP*',
 'YCP',
 'TRADE',
 'VALUE (mn)',
 'VOLUME']

In [1]:
def loadFixedStockData():
    dataFile = 'stockData.csv'
    data = pd.read_csv(dataFile,parse_dates=True)
    data = data.rename(columns={'DATE':'Date',
    'TRADING CODE':'TradingCode',
    'LTP*':'LTP',
    'HIGH':'Adj. High',
    'LOW':'Adj. Low',
    'OPENP*':'Adj. Open',
    'CLOSEP*':'Adj. Close',
    'YCP':'YCP',
    'TRADE':'Trade',
    'VALUE (mn)':'Value',
    'VOLUME':'Adj. Volume'})
    data['Date'] = data['Date'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))
    data['Adj. Volume'] = data['Adj. Volume'].apply(lambda x: float(x.replace(',','')))
    data = data.set_index('Date')
    data.fillna(0.1, inplace=True)
    return data
quandl.ApiConfig.api_key = "qWcicxSctVxrP9PhyneG"
allData = loadFixedStockData() #quandl.get('WIKI/'+stock)
dataLength = 251
allDataLength = len(allData)
firstDataElem = math.floor(random.random()*(allDataLength-dataLength))
mlData = allData[0:firstDataElem+dataLength]

def FormatForModel(dataArray):
    dataArray = dataArray[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
    dataArray['HL_PCT'] = (dataArray['Adj. High'] - dataArray['Adj. Close']) / dataArray['Adj. Close'] * 100.0
    dataArray['PCT_change'] = (dataArray['Adj. Close'] - dataArray['Adj. Open']) / dataArray['Adj. Open'] * 100.0
    dataArray = dataArray[['Adj. Close', 'HL_PCT', 'PCT_change','Adj. Volume']]
    dataArray = dataArray.replace(0,0.1)
    dataArray = dataArray.replace([np.inf,-np.inf],np.nan)
    dataArray.fillna(0.1, inplace=True)
    return dataArray

mlData = FormatForModel(mlData)

forecast_col = 'Adj. Close'
forecast_out = int(math.ceil(0.12*dataLength))


mlData['label'] = mlData[forecast_col].shift(-forecast_out)
mlData.dropna(inplace=True)

X = np.array(mlData.drop(['label'],1))
np.max(X)

NameError: name 'quandl' is not defined

In [4]:
dataFile = 'stockData.csv'
data = pd.read_csv(dataFile,parse_dates=True)
data = data.rename(columns={'DATE':'Date',
'TRADING CODE':'TradingCode',
'LTP*':'LTP',
'HIGH':'Adj. High',
'LOW':'Adj. Low',
'OPENP*':'Adj. Open',
'CLOSEP*':'Adj. Close',
'YCP':'YCP',
'TRADE':'Trade',
'VALUE (mn)':'Value',
'VOLUME':'Adj. Volume'})
data['Date'] = data['Date'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))
data['Adj. Volume'] = data['Adj. Volume'].apply(lambda x: float(x.replace(',','')))
data = data.set_index('Date')
data.fillna(0.1, inplace=True)
data = data.sort_index()
data.head()

,#,TradingCode,LTP,Adj. High,Adj. Low,Adj. Open,Adj. Close,YCP,Trade,Value,Adj. Volume
Date,,,,,,,,,,,
2017-01-01,528,SQURPHARMA,248.2,249.5,248.0,249.5,248.2,249.1,"1,031",73.705,296729.0
2017-01-02,527,SQURPHARMA,249.0,250.0,248.1,248.5,249.4,248.2,"1,344",104.817,421178.0
2017-01-03,526,SQURPHARMA,251.5,251.5,248.8,250.0,250.9,249.4,"1,314",136.777,546260.0
2017-01-04,525,SQURPHARMA,253.0,254.3,250.0,250.9,253.7,250.9,"1,608",129.563,514003.0
2017-01-05,524,SQURPHARMA,252.5,254.0,252.1,253.0,253.0,253.7,"1,335",99.740,393932.0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 528 entries, 2019-02-24 to 2017-01-01
Data columns (total 4 columns):
Adj. Close     528 non-null float64
HL_PCT         528 non-null float64
PCT_change     528 non-null float64
Adj. Volume    528 non-null float64
dtypes: float64(4)
memory usage: 20.6 KB


In [48]:
type(data)

pandas.core.frame.DataFrame

In [9]:
quandl.ApiConfig.api_key = "qWcicxSctVxrP9PhyneG"
tripStockData = quandl.get('WIKI/TRIP')

In [14]:
tripStockData.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2011-12-07,27.50,27.76,24.000,27.45,1413700.0,0.0,1.0,27.50,27.76,24.000,27.45,1413700.0
2011-12-08,26.06,27.78,25.759,26.55,719600.0,0.0,1.0,26.06,27.78,25.759,26.55,719600.0
2011-12-09,27.93,28.00,26.700,27.91,716200.0,0.0,1.0,27.93,28.00,26.700,27.91,716200.0
2011-12-12,28.28,28.28,27.000,27.45,393300.0,0.0,1.0,28.28,28.28,27.000,27.45,393300.0
2011-12-13,28.53,28.53,27.010,27.50,452500.0,0.0,1.0,28.53,28.53,27.010,27.50,452500.0


In [13]:
tripStockData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1584 entries, 2011-12-07 to 2018-03-27
Data columns (total 12 columns):
Open           1584 non-null float64
High           1584 non-null float64
Low            1584 non-null float64
Close          1584 non-null float64
Volume         1584 non-null float64
Ex-Dividend    1584 non-null float64
Split Ratio    1584 non-null float64
Adj. Open      1584 non-null float64
Adj. High      1584 non-null float64
Adj. Low       1584 non-null float64
Adj. Close     1584 non-null float64
Adj. Volume    1584 non-null float64
dtypes: float64(12)
memory usage: 160.9 KB


In [46]:
type(tripStockData)

pandas.core.frame.DataFrame

In [57]:
list(tripStockData.columns.values)

['Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'Ex-Dividend',
 'Split Ratio',
 'Adj. Open',
 'Adj. High',
 'Adj. Low',
 'Adj. Close',
 'Adj. Volume']

In [27]:
allData = quandl.get('WIKI/TRIP')
dataLength = 251
allDataLength = len(allData)
firstDataElem = math.floor(random.random()*(allDataLength-dataLength))
mlData = allData[0:firstDataElem+dataLength]

def FormatForModel(dataArray):
    dataArray = dataArray[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
    dataArray['HL_PCT'] = (dataArray['Adj. High'] - dataArray['Adj. Close']) / dataArray['Adj. Close'] * 100.0
    dataArray['PCT_change'] = (dataArray['Adj. Close'] - dataArray['Adj. Open']) / dataArray['Adj. Open'] * 100.0
    dataArray = dataArray[['Adj. Close', 'HL_PCT', 'PCT_change','Adj. Volume']]
    dataArray.fillna(0.1, inplace=True)
    dataArray.replace(0,0.1)
    return dataArray

mlData = FormatForModel(mlData)

forecast_col = 'Adj. Close'
forecast_out = int(math.ceil(0.12*dataLength))


mlData['label'] = mlData[forecast_col].shift(-forecast_out)
mlData.dropna(inplace=True)

X = np.array(mlData.drop(['label'],1))
np.max(X)

c:\users\hp\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
c:\users\hp\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


12923400.0